#VAST Challenge 2018: MC1 
<hr>
## Cameron Henkel and Colin Scruggs
Credit to [Dr. Jaron Collis](https://github.com/jaron/deep-listening) for providing much of this notebook's source code 

**Import necessary libraries:**

In [0]:
# Uncomment if using a Colaboratory hosted runtime
#!pip install -q librosa
# from google.colab import files

import glob
import os
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Sound Classification & Feature Extraction
(local runtime only)

In [0]:
# Establish variables for parent and save directories 
parent_dir = './notebook_resources/all-birds-wav'
save_dir = './notebook_resources/bird_rnn_features_labels'

In [0]:
# Create dataframe from csv containing metadata about each recording
datafile = "./public/data/AllBirdsv4-refined.csv"
df = pd.read_csv(datafile)

# Create a copy of the dataframe with entries of quality A, B, or C
sample_DF = df.copy()
sample_DF = sample_DF[sample_DF['Quality'].isin(['A', 'B', 'C'])]

# Append a species id column to the dataframe for labeling purposes
Species_ID = sample_DF.English_name.factorize()[0]

sample_DF['Species_ID'] = Species_ID

# ------------------------------------ #

# Fully remove invalid files
sample_DF = sample_DF.drop(sample_DF.loc[sample_DF['File ID']==244588].index)
sample_DF = sample_DF.drop(sample_DF.loc[sample_DF['File ID']==244587].index)

# Remove long audio files and re-add split versions
removed_ID = [371463, 327002, 171698, 244585, 325435, 132146, 179051, 256836, 161306, 192536, 306348, 305896, 153038, 206183,
             206157, 210466, 163245, 185518]

# Modify the csv to include the new file ids of each split audio file
for file in os.listdir("./notebook_resources/all-birds-wav/bird_splits"):
    file = file[:-4]
    orig_ID = file.split("-")[0]
    orig_index = sample_DF.loc[sample_DF['File ID']==int(orig_ID)].index.tolist()
    temp_species_ID = sample_DF.at[orig_index[0],'Species_ID']
    sample_DF = sample_DF.append({'File ID': file,'Species_ID': temp_species_ID}, ignore_index=True)

# Remove original long file rows from dataframe to not conflict with the split ones just added
for i, value in enumerate(removed_ID):
    sample_DF = sample_DF.drop(sample_DF.loc[sample_DF['File ID']==value].index)
    
# ------------------------------------ #

# Grab column of file IDs from bird metadata
file_ID = sample_DF['File ID'] 

In [0]:
# Define the windowing function of the FFT
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size // 2)

# Extract individual audio features from each recording using MFCC & FFT
def extract_features(parent_dir,file_ext=".wav",bands = 20, frames = 41):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = []
    ref_ID = []
    count_max = len(sample_DF)
    # Iterate through every File ID in the dataframe which corresponds to an audio recording
    for count, id in enumerate(file_ID):
        fn = os.path.join(parent_dir, str(id) + file_ext)
        print('Processing %d/%d: %s ...' % (count, count_max, id))
        
        # Load audio clip into librosa
        sound_clip,s = librosa.load(fn)
        
        # Normalize audio
        sound_clip_normalized = librosa.util.normalize(sound_clip)
        
        # Split audio time series into non-silent sections
        split_array = librosa.effects.split(sound_clip_normalized, top_db=10, frame_length=32768, hop_length=256)
        split_chirps = []
        
        # Reform a single time series from the separate non-silent components
        for i,interval in enumerate(split_array):
            split_chirps.append(librosa.effects.remix(sound_clip_normalized, intervals=[interval]))

        # Set classification labels for later machine learning using corresponding Species ID
        species_id =  sample_DF.loc[sample_DF['File ID'] == id, 'Species_ID'].tolist()[0]
        label = species_id
        
        # Loop through each individual audio segment (bird chirp) and apply MFCC to generate features
        for i, audio_segment in enumerate(split_chirps):
            # Uncomment to save individual chirps - consumes a lot of storage space
            # save_dir = './bird_rnn_split_wavs'
            # file_path = save_dir + "/" + str(id) + "-" + str(i) + ".wav"
            # librosa.output.write_wav(file_path, audio_segment, sr=s)
            for (start,end) in windows(audio_segment,window_size):
                start = int(start)
                end = int(end)
                if(len(audio_segment[start:end]) == window_size):
                    signal = audio_segment[start:end]
                    mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                    mfccs.append(mfcc)
                    labels.append(label)
                    ref_ID.append(id)
    features = np.asarray(mfccs).reshape(len(mfccs),bands,frames)
    return np.array(features), np.array(labels,dtype = np.int), np.array(ref_ID)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [0]:
# Generate features and labels from audio files
features, labels, ref_ID = extract_features(parent_dir)
labels = one_hot_encode(labels)

In [0]:
# Use this to save the features and labels to external files for later reuse
def save_folds(data_dir):
    print("Features = ", features.shape)
    print("Labels = ", labels.shape)
        
    feature_file = os.path.join(data_dir, "Features" + '_x.npy')
    labels_file = os.path.join(data_dir, "Labels" + '_y.npy')
    ref_file = os.path.join(data_dir, "References" + '_y.npy')
    np.save(feature_file, features)
    print("Saved " + feature_file)
    np.save(labels_file, labels)
    print("Saved " + labels_file)
    np.save(ref_file, ref_ID)
    print("Saved " + ref_file)

def assure_path_exists(path):
    mydir = os.path.join(os.getcwd(), path)
    if not os.path.exists(mydir):
        os.makedirs(mydir)
        
# Uncomment this to recreate and save the feature vectors
assure_path_exists(save_dir)
save_folds(save_dir)

## Reload Features and Labels

In [0]:
# This method accesses and returns the Features and Labels (.npy)
def driveLoad(base_dir):
    print("Loading features...")
    feature_file = os.path.join(base_dir+"Features_x.npy")
    labels_file = os.path.join(base_dir+"Labels_y.npy")
    loaded_features = np.load(feature_file)
    loaded_labels = np.load(labels_file)
    
    features = loaded_features
    labels = loaded_labels
    
    print("Done")
    return features, labels

### Reload from Google Drive
Must be running in Colaboratory hosted runtime

#### FUSE Google Drive
This section is used to establish a connection between our shared Google Drive folder that contains the extracted features and labels .npy files and then load them as local variables.   

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# Check that Features and Labels are in correct folder
!ls drive/all_birds_features_labels/

#### Reload features and labels with Drive

In [0]:
# Use load_folds to initialize the features and labels objects from Google Drive

features, labels = driveLoad("drive/all_birds_features_labels/")

### Reload from Local System
Must be running in local runtime

In [0]:
# Use load_folds to initialize the features and labels objects from a local filesystem

features, labels = driveLoad("./notebook_resources/bird_rnn_features_labels/")

## Training a Recurrent Neural Network with Keras and TensorFlow

In [0]:
# Set random seeds for Tensorflow and Numpy
tf.set_random_seed(0)
np.random.seed(0)

Split loaded data into train/test/validation according to model

In [0]:
import random

# Extract random 19 features for predicting
prediction_features = [] # One feature from each species

# Create splits and assign train, validation, and test data
train_test_split = np.random.rand(len(features)) < 0.80 # training and testing split

train_x = features[train_test_split]
train_y = labels[train_test_split]

# From the remaining ~20 percent of features/labels, create a new split for testing and validation data

tstv_features = features[~train_test_split]
tstv_labels = labels[~train_test_split]

train_validate_split = np.random.rand(len(tstv_features)) <0.98 # testing and validation split

# Assign validation features and labels (~98%)
valid_x = tstv_features[train_validate_split]
valid_y = tstv_labels[train_validate_split]

# Assign testing features and labels (~2%)
test_x = tstv_features[~train_validate_split]
test_y = tstv_labels[~train_validate_split]

prediction_features=test_x # For later use in 'Test Model' section
print("Training and validation data loaded.")


### Generate Model
These are three machine learning models we used to generate predictions during our research.  
We found the RNN LSTM model and the GRU model to be the most accurate.
The GRU model generally finished training faster than the RNN LSTM.  
  
    
References:  
[RNN, LSTM, and GRU tutorial](https://jhui.github.io/2017/03/15/RNN-LSTM-GRU/)  
[Introduction to Long Short Term Memory](https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/)  
[Understanding GRU networks](https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be)

In [0]:
from keras.models import Sequential
data_dim = 41
timesteps = 20
num_classes = 19

#### RNN LSTM Model 


In [0]:
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

# returns a sequence of vectors of dimension 256
model.add(LSTM(512, return_sequences=True, input_shape=(timesteps, data_dim)))  
model.add(Dropout(0.15))

# returns a sequence of vectors of dimension 256
model.add(LSTM(512, return_sequences=True))  
model.add(Dropout(0.25))

# return a single vector of dimension 128
model.add(LSTM(256))  
model.add(Dropout(0.35))

# apply softmax to output
model.add(Dense(num_classes, activation='softmax'))

# compile the model for multi-class classification
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# a stopping function to stop training before we excessively overfit to the training set
earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')
model.fit(train_x, train_y, batch_size=128, epochs=20, callbacks=[earlystop],validation_data=(valid_x, valid_y))

#### Stacked LSTM Model

In [0]:
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

batch_size = 64
epochs = 15

# Stacked LSTM
# 2nd to last model on https://keras.io/getting-started/sequential-model-guide/
model = Sequential()
model.add(LSTM(128, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(128, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(128))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))


# Stateful Stacked LSTM (?)
'''
model.add(LSTM(32, return_sequences=True, stateful=True,
               input_shape=(timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(10, activation='softmax'))
'''

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# a stopping function to stop training before we excessively overfit to the training set
earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')

model.fit(train_x, train_y,
          batch_size=batch_size, epochs=epochs, callbacks=[earlystop],
          validation_data=(valid_x, valid_y))

#### GRU Model

In [0]:
from keras.layers import GRU, Dense, Dropout
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

model = Sequential()

#1024
model.add(GRU(512, return_sequences=True, input_shape=(timesteps, data_dim)))  
model.add(Dropout(0.15))
#1024
model.add(GRU(512, return_sequences=True))  
model.add(Dropout(0.25))

model.add(GRU(512))  
model.add(Dropout(0.35))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')
print("You're here")
model.fit(train_x, train_y, batch_size=256, epochs=2, callbacks=[earlystop], validation_data=(valid_x, valid_y))

### Test Model

In [0]:
# Match species names to correct classification labels
species_names = ["Rose-crested Blue Pipit", "Blue-collared Zipper", "Bombadil", "Broad-winged Jojo", "Canadian Cootamum", 
                      "Carries Champagne Pipit", "Darkwing Sparrow", "Eastern Corn Skeet", "Green-tipped Scarlet Pipit", 
                      "Lesser Birchbeere", "Orange Pine Plover", "Ordinary Snape", "Pinkfinch", "Purple Tooting Tout", 
                      "Qax", "Queenscoat", "Bent-beak Riffraff", "Scrawny Jay", "Vermillion Trillian"]

model_prediction_species = []

for oneHot in test_y:
    uniqueValues, uniqueIndexes = np.unique(oneHot, return_index=True)
    
    model_prediction_species.append(species_names[uniqueIndexes[1]])
    #print ("Index: " + str(index) + " | " + species_names[index])

In [0]:
# Create predictions for each of the sound classes

# Variables used to calculate correct percentanges 
total_correct = 0
correct_by_species = []
species_correct = 0;
species_total = 0;
current_species = "Rose-crested Blue Pipit"

# Loop over each feature set from test data
for s in range(len(model_prediction_species)):
    
    # Handles correct percentages per species
    if (current_species != model_prediction_species[s]):
        percent = float(species_correct) / species_total
        correct_by_species.append(str(current_species) + ": " + str(percent) + " (" + str(species_correct) + "/" + str(species_total) + ")")
        species_correct = 0
        species_total = 0
        
    # Grab ground truth for current feature set to be tested against
    current_species = model_prediction_species[s]
    #print ("\n----- ", current_species, "-----")
    
    # Load prediction_features for current_species
    predict_x = prediction_features[s]
    predict_x = np.expand_dims(predict_x, axis=0)
    predictions = model.predict(predict_x)

    # If no prediction was able to be generated, print so
    if len(predictions) == 0: 
        print ("No prediction")
    
    # If not, print the top 3 guesses
    ind = predictions[0].argsort()[-3:][::-1]
    #print ("Top guess: ", species_names[ind[0]], " (",round(predictions[0,ind[0]],3),")")
    #print ("2nd guess: ", species_names[ind[1]], " (",round(predictions[0,ind[1]],3),")")
    #print ("3rd guess: ", species_names[ind[2]], " (",round(predictions[0,ind[2]],3),")")
    
    # Accumulate number of correct guesses for percentage calculations
    if (species_names[ind[0]] == current_species):
        total_correct += 1
        species_correct += 1
    species_total += 1

# Print final results
percentCorrect = (float(total_correct) / len(model_prediction_species))
print ("\n----- Percent correct -----")
print (percentCorrect)
print (correct_by_species)
for i in correct_by_species: # Print out the percentage correct for each classifier
    print(i)

### Load/Save Model

In [0]:
# Saves model to local filesystem root directory

fileName = "insert desired name here"
save_model_name = './notebook_resources/'+fileName + ".h5"
model.save(save_model_name)

**Save Model - Hosted runtime**

In [0]:
# Hosted runtime only - must run previous cell to save model to the Hosted virtual machine's root directory, then the line below downloads that file to the user's local machine

#files.download(save_model_name)
#print(save_model_name + " has been saved.")

**Load Model - Local Runtime**

In [0]:
from keras.models import load_model
model_file = os.path.join('./notebook_resources/'+fileName+".h5")
model = load_model(model_file)

**Load Model - Hosted Runtime**  
Only use if Google Drive has been FUSED and you're on a hosted runtime

In [0]:
!ls "drive/tensorflow_models/"

In [0]:
from keras.models import load_model

model_name = "GRU_19_19.h5"
model = load_model(os.path.join("drive/tensorflow_models/"+model_name))

print(model_name + " has been loaded.")

## Generating Kasios Predictions

### Waveform/spectrogram visualization methods

**Import necessary libraries:**

In [0]:
# Uncomment if using a Colaboratory hosted runtime
# !pip install -q librosa
import librosa
from librosa import display
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('seaborn-pastel')

In [0]:
# Defines method for creating a waveform visualization of any sound file
def visualizeWave(sound_clip,s,file_ID): 
    plt.figure(figsize=(20,4))
    plt.subplot(1, 1, 1)
    librosa.display.waveplot(sound_clip, sr=s)
    plt.title(file_ID)
    plt.show()

In [0]:
# Defines method for creating a spectrogram visualization of any sound file
def visualizeSpecgram(sound_clip,file_ID):
    fig = plt.figure(figsize=(20,4))
    plt.subplot(1,1,1)
    specgram(np.array(sound_clip), Fs=22050, cmap="hot")
    plt.title(file_ID)
    #plt.suptitle('Figure 2: Spectrogram',x=0.5, y=0.95,fontsize=18)
    #plt.colorbar(mappable=mappable,ax=ax)
    plt.show()

### Feature extraction

In [0]:
# Define the windowing function of the FFT
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size // 2)

# Extract individual audio features from each Kasios recording using MFCC & FFT
def extract_features(parent_dir,file_ext=".wav",bands = 20, frames = 41):
    window_size = 512 * (frames - 1)
    mfccs = []
    file_features_indexes = [0]
    chirp_features_indexes = [0]
    
    for id in range(1,16): # loop through 1-15 (file names of provided Kasios files)
        print("Processing file " + str(id) + "...")
        fn = os.path.join(parent_dir, str(id) + file_ext)
        sound_clip,s = librosa.load(fn)
        
        # --- VISUALIZE RAW AUDIO FILE ---
        #visualizeWave(sound_clip, s, id)
        #visualizeSpecgram(sound_clip, id)
        
        split_array = librosa.effects.split(sound_clip, top_db=10, frame_length=32768, hop_length=256)
        #print(split_array)
        #print(split_array/22050)
        split_chirps = []
        
        for i,interval in enumerate(split_array):
            split_chirps.append(librosa.effects.remix(sound_clip, intervals=[interval]))
        
        # --- VISUALIZE FIRST SPLIT CHIRP ---
        #visualizeWave(split_chirps[0], s, id)
        #visualizeSpecgram(split_chirps[0], id)
        
        # --- VISUALIZE REMIXED SPLIT ---
        remixed_split = librosa.effects.remix(sound_clip, intervals=librosa.effects.split(sound_clip, top_db=10, frame_length=32768, hop_length=256))
        #visualizeWave(remixed_split, s, id)
        #visualizeSpecgram(remixed_split, id)
        
        for i, audio_segment in enumerate(split_chirps):
            # --- VISUALIZE EACH CHIRP ---
            #visualizeWave(audio_segment, s, (id,i))
            #visualizeSpecgram(audio_segment, id)
            
            # save_dir = './bird_rnn_split_wavs'
            # file_path = save_dir + "/" + str(id) + "-" + str(i) + ".wav"
            # librosa.output.write_wav(file_path, audio_segment, sr=s)
            for (start,end) in windows(audio_segment,window_size):
                start = int(start)
                end = int(end)
                #print(start, end, len(sound_clip))
                if(len(audio_segment[start:end]) == window_size):
                    signal = audio_segment[start:end]
                    mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                    mfccs.append(mfcc)
            chirp_features_indexes.append(len(mfccs))
        file_features_indexes.append(len(mfccs))
        print("File " + str(id) + " done.\n")
        

    features = np.asarray(mfccs).reshape(len(mfccs),bands,frames)
    return np.array(features), file_features_indexes, chirp_features_indexes

In [0]:
# Colaboratory hosted runtime / FUSED Drive mode
#data_dir = "drive/test-birds-from-kasios-cleaned/"

# Local runtime / local filesystem mode
data_dir = "./notebook_resources/test-birds-from-kasios-cleaned/"

kasios_features, file_indexes, chirp_indexes = extract_features(data_dir)
#print(kasios_features, file_indexes, chirp_indexes)

### Produce predictions

In [0]:
prediction_species = ["Rose-crested Blue Pipit", "Blue-collared Zipper", "Bombadil", "Broad-winged Jojo", "Canadian Cootamum", 
                      "Carries Champagne Pipit", "Darkwing Sparrow", "Eastern Corn Skeet", "Green-tipped Scarlet Pipit", 
                      "Lesser Birchbeere", "Orange Pine Plover", "Ordinary Snape", "Pinkfinch", "Purple Tooting Tout", 
                      "Qax", "Queenscoat", "Bent-beak Riffraff", "Scrawny Jay", "Vermillion Trillian"]

In [0]:
# Setup csv column structure for aggregate predictions
predictions_DF = pd.DataFrame(columns=['Model_Name', 'ID', 'first_guess', 'first_confidence', 'second_guess', 'second_confidence', 'third_guess', 'third_confidence'])
print(predictions_DF)

In [0]:
model_name = "GRU"

In [0]:
# Create predictions for each of the bird species

for s in range(len(file_indexes) - 1): # s: 0->14
    print ("\n----- File ", s+1, "-----")
    
    # Load prediction_features for s species
    if (s == 0): # if check to prevent overlap  
        predict_x = kasios_features[file_indexes[s]: file_indexes[s+1]]
        # print(file_indexes[s], file_indexes[s+1])
    else:
        predict_x = kasios_features[file_indexes[s]+1: file_indexes[s+1]]
        # print(file_indexes[s]+1, file_indexes[s+1])
    
    predictions = model.predict(predict_x)

    if len(predictions) == 0: 
        print ("No prediction")
    
    ind = predictions[0].argsort()[-3:][::-1]

    print ("Top guess: ", prediction_species[ind[0]], " (",round(predictions[0,ind[0]],3),")")
    print ("2nd guess: ", prediction_species[ind[1]], " (",round(predictions[0,ind[1]],3),")")
    print ("3rd guess: ", prediction_species[ind[2]], " (",round(predictions[0,ind[2]],3),")")
    predictions_DF = predictions_DF.append({'Model_Name': model_name,'ID': s+1, 'first_guess': prediction_species[ind[0]], 'first_confidence': round(predictions[0,ind[0]],3), 'second_guess': prediction_species[ind[1]], 'second_confidence': round(predictions[0,ind[1]],3), 'third_guess': prediction_species[ind[2]], 'third_confidence': round(predictions[0,ind[2]],3)}, ignore_index=True) 

print (predictions_DF);

In [0]:
# Save prediction data from tests to csv
test = pd.DataFrame.to_csv(predictions_DF, path_or_buf='./public/data/aggregate_predictions.csv', encoding='utf-8', index=False)

# Download csv from Hosted Runtime
#files.download('aggregate_predictions.csv')